## Базовая функциональность

In [1]:
import numpy as np
import pandas as pd

### Переиндексация
Для объектов pandas важен метод `reindex`, т.е. возможность создания нового объекта, в котором данные переупорядочены в соответствии с новым индексом.

In [2]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
print(obj, '\n')

obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
print(obj2)

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64 

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64


Для упорядоченных данных, например временных рядов, иногда желательно произвести интерполяцию, или восполнение отсутствующих значений в процессе переиндексации. Это позволяет сделать параметр `method`; так, если задать для него значение `ffill`, то вместо отсутствующих значений будут подставлены значения из ближайшей предшествующей строки, имеющейся в индексе:

In [3]:
obj3 = pd.Series(['blue', 'red', 'green'], index=[0, 2, 4])
print(obj3, '\n')
obj4 = obj3.reindex(np.arange(6), method='ffill')
print(obj4)

0     blue
2      red
4    green
dtype: object 

0     blue
1     blue
2      red
3      red
4    green
5    green
dtype: object


В случае объекта DataFrame метод `reindex` может изменять строки, столбцы или то и другое. Если передать ему просто последовательность, то в результирующем объекте переиндексируются строки:

In [ ]:
frame = pd.DataFrame(np.arange(9).reshape(3, 3), 
                     index=['a', 'c', 'd'], 
                     columns=['Ohio', 'Texas', 'California'])

frame 

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [6]:
frame2 = frame.reindex(index=['a', 'b', 'c', 'd'])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


Столбцы можно переиндексировать, задав ключевое слово `columns`:

In [9]:
states = ['Texas', 'Utah', 'California']
frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


Другой способ переиндексации по конкретной оси:

In [10]:
frame.reindex(states, axis='columns') 

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8



| Параметр | Описание |
|----------|----------|
| **labels** | То же, что и `index` (устаревшее) |
| **index** | Использовать переданную последовательность в качестве новых меток индекса |
| **columns** | Использовать переданную последовательность в качестве новых меток столбцов |
| **axis** | По какой оси переиндексировать: "index" (строки) или "columns" (столбцы). По умолчанию подразумевается "index". Можно чередовать reindex(index=new_labels) и reindex(columns=new_labels) |
| **method** | Метод интерполяции (восполнения): "ffill" (прямое восполнение – по предыдущей строке) или "bfill" (прямое восполнение – по следующей строке) |
| **fill_value** | Значение, которое должно подставляться вместо отсутствующих значений, появляющихся в результате переиндексации. Если нужно, чтобы для отсутствующих меток принималось значение NaN, то задавайте fill_value="missing" |
| **limit** | При прямом или обратном восполнении максимальная длина восполняемой лакуны (выраженная числом элементов) |
| **tolerance** | При прямом или обратном восполнении максимальная длина восполняемой лакуны (выраженная абсолютным расстоянием) |
| **level** | Сопоставить с простым объектом Index на указанном уровне MultiIndex, иначе выбрать подмножество |
| **copy** | Если True, то всегда копировать данные, даже если новый индекс эквивалентен старому. Если False, то не копировать данные, когда индексы эквивалентны |


*Лакуна - непрерывный участок отсутствующих данных, который нужно обработать*

Еще один популярный способ переиндексировать - с помощью оператора `loc`. Он работает, только если все метки нового индекса уже присутствуют в объекте DataFrame (тогда как `reindex` вставляет отсутствующие данные для новых меток): 

In [11]:
frame.loc[['a', 'd', 'c'], ['California', 'Texas']]

,California,Texas
a,2,1
d,8,7
c,5,4


### Удаление элементов из оси
Чтобы удалить один или несколько элементов из оси без использования сложных конструкций, стоит использовать `drop`. Этот метод возвращает новый объект, в котором указанные значения удалены из оси:

In [15]:
obj = pd.Series(np.arange(5), index=list('abcde'))
obj

a    0
b    1
c    2
d    3
e    4
dtype: int64

In [19]:
new_obj = obj.drop(index=['c', 'd'])
new_obj

a    0
b    1
e    4
dtype: int64

В случае DataFrame указанные в аргументе `index` значения можно удалить из любой оси:

In [20]:
data = pd.DataFrame(np.arange(16).reshape(4, 4), 
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


Если вызвать `drop`, указав последовательность меток, то будут удалены строки с такими метками.  
Чтобы удалить столбцы, используйте именованный аргумент `columns`.  
Столбцы можно удалить также, передав параметры `axis=1` (как в NumPy) или `axis="columns"`.

### Доступ по индексу, выборка и фильтрация
Доступ по индексу к объекту Series (`obj[...]`) работает так же, как и для массивов NumPy:

In [21]:
obj = pd.Series(np.arange(4), index=list('abcd'))
obj

a    0
b    1
c    2
d    3
dtype: int64

In [31]:
print('.b => ', obj.b)
print("[c] => ", obj['c'])
print("['b', 'a', 'd'] =>\n", obj[['b', 'a', 'd']])
print('[1, 3] =>\n', obj.iloc[[1, 3]])
print('[obj < 2] =>\n', obj[obj < 2])

.b =>  1
[c] =>  2
['b', 'a', 'd'] =>
 b    1
a    0
d    3
dtype: int64
[1, 3] =>
 b    1
d    3
dtype: int64
[obj < 2] =>
 a    0
b    1
dtype: int64



*Предпочтительнее выбирать значения из индекса с помощью специальных операторов `loc` и `iloc`.*

In [ ]:
obj.loc[['a', 'b', 'c']] # loc для меток

a    0
b    1
c    2
dtype: int64

In [ ]:
obj.iloc[[0, 1, 2]] # iloc для числовых индексов

a    0
b    1
c    2
dtype: int64

С помощью меток можно также производить вырезание, но оно отличается от обычного вырезания в Python тем, что **конечная точка включается**:

In [38]:
obj.loc['b' : 'c']

b    1
c    2
dtype: int64

Обращение по индексу к DataFrame предназначено для получения одного или нескольких столбцов путем задания одного значения или последовательности:

In [40]:
data = pd.DataFrame(np.arange(16).reshape(4, 4),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [45]:
data['two']

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int64

In [46]:
data[['three', 'one']]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


In [47]:
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [48]:
data[data['three'] > 5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


### Выборка из DataFrame с помощью `loc` и `iloc`
Как и Series, объект DataFrame имеют специальные атрибуты `loc` и `iloc` для индексирования метками и целыми числами соответственно. Можно выбрать подмножество и строк, и столбцов одновременно.

In [49]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [50]:
data.loc['Colorado']

one      4
two      5
three    6
four     7
Name: Colorado, dtype: int64

In [51]:
data.loc[['Colorado', 'Utah']]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11


Оператор `loc` позволяет выбрать одновременно строки и столбцы:

In [52]:
data.loc['Colorado', ['two', 'three']]

two      5
three    6
Name: Colorado, dtype: int64

Аналогично работает и `iloc`:

In [54]:
data.iloc[[1, 2], [3, 0, 1]]

,four,one,two
Colorado,7,4,5
Utah,11,8,9


Обе функции индексирования работают не только с одиночными метками ил списками меток, но и со срезами:

In [55]:
data.loc[:'Utah', 'two']

Ohio        1
Colorado    5
Utah        9
Name: two, dtype: int64

In [56]:
data.iloc[:, :3][data.three > 5]

,one,two,three
Colorado,4,5,6
Utah,8,9,10
New York,12,13,14


#### Варианты доступа к объекту DataFrame
| Метод | Описание |
|-------|----------|
| **df[column]** | Выбрать один столбец или последовательность столбцов из DataFrame. Частные случаи: булев массив (фильтрация строк), срез (вырезание строк) или булев DataFrame (установка значений в позициях, удовлетворяющих некоторому критерию) |
| **df.loc[rows]** | Выбрать одну строку или подмножество строк из DataFrame по метке |
| **obj.loc[:, cols]** | Выбрать один столбец или подмножество столбцов по метке |
| **df.iloc[rows, cols]** | Выбрать строки и столбцы по целочисленной позиции |
| **df.iloc[rows]** | Выбрать одну строку или подмножество строк из DataFrame по целочисленной позиции |
| **obj.iloc[:, cols]** | Выбрать один столбец или подмножество столбцов по целочисленной позиции |
| **df.loc[rows, cols]** | Выбрать строки и столбцы по метке |
| **df.at[row, col]** | Выбрать одно скалярное значение по меткам строки и столбца |
| **df.iat[row, col]** | Выбрать одно скалярное значение по целочисленным позициям строки и столбца |
| **reindex()** | Выбрать строки или столбцы по меткам |

### Арифметические операции и выравнивание данных
pandas может сильно упростить работу с объектами, имеющими различные индексы. Так, если при сложении двух объектов в одном индексе обнаруживается значение, отсутствующее в другом, то результирующий индекс будет объединением исходных.

In [62]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s2 = pd.Series([-2.1, 3.6, -1.5, 4., 3.1], index=['a', 'c', 'e', 'f', 'g'])

print(s1, '\n------------\n', s2)

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64 
------------
 a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64


In [63]:
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

Вследствие внутреннего выравнивания данных образуются значения `NaN` в позициях, для которых не нашлось соответственной пары. Эти значения распространяются на последующие операции.

В случае DataFrame выравнивание производится как для строк, так и длля столбцов:

In [64]:
df1 = pd.DataFrame(np.arange(9).reshape(3, 3), 
                   columns=['b', 'c', 'd'],
                   index=['Ohio', 'Texas', 'Colorado'])

df2 = pd.DataFrame(np.arange(12).reshape(4, 3),
                   columns=['b', 'd', 'e'],
                   index=['Utah', 'Ohio', 'Texas', 'Oregon'])

print(df1, '\n------------\n', df2)

          b  c  d
Ohio      0  1  2
Texas     3  4  5
Colorado  6  7  8 
------------
         b   d   e
Utah    0   1   2
Ohio    3   4   5
Texas   6   7   8
Oregon  9  10  11


При сложении этих объектов получается DataFrame, индекс и столбцы которого являются объединениями индексов и столбцов слагаемых:

In [65]:
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


Поскольку столбцы `c` и `e` не встречаются в обоих DataFrame сразу, то в результирующем объекте они помечены как отсутствующие. То же самое относится к строкам и меткам, не встречающихся в обоих объектах.  
Если сложить объекты DataFrame, не имеющие обоих столбцов и строк, то результат будет содержать только значения `NaN`:

In [68]:
df1 = pd.DataFrame({'A': [1, 2]})
df2 = pd.DataFrame({'B': [3, 4]})

df1 + df2

,A,B
0,NaN,NaN
1,NaN,NaN


#### Восполнение значений в арифметических методах
При выполнении арифметических операций с объектами, проиндексированными по-разному, иногда желательно поместить специальное значение, например 0, в позиции операнда, которым в другом операнде соответствует отсутствующая позиция. 

In [69]:
df1 = pd.DataFrame(np.arange(12).reshape(3, 4), 
                   columns=list('abcd'))
df2 = pd.DataFrame(np.arange(20).reshape(4, 5), 
                   columns=list('abcde'))
df2.loc[1, 'b'] = np.nan

print(df1, '\n------------\n', df2)

   a  b   c   d
0  0  1   2   3
1  4  5   6   7
2  8  9  10  11 
------------
     a     b   c   d   e
0   0   1.0   2   3   4
1   5   NaN   7   8   9
2  10  11.0  12  13  14
3  15  16.0  17  18  19


In [70]:
df1 + df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,NaN,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


Чтобы заполнить все `NaN` как некоторые значения, нужно использовать `fill_value`:

In [71]:
df1.add(df2, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,5.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


В таблице ниже представлены методы объектов Series и DataFrame. У каждого из них имеется вариант, начинающийся буквой **r** (`a.sub(b) == b.rsub(a) == a - b`), в котором аргументы переставлены местами. Например следующие операции дадут один и тот же результат:  
`1 / df1 == df1.rdiv(1)`

| Метод | Описание | Оператор |
|-------|----------|----------|
| **add**, **radd** | Сложение | `+` |
| **sub**, **rsub** | Вычитание | `-` |
| **div**, **rdiv** | Деление | `/` |
| **floordiv**, **rfloordiv** | Деление с отсечением | `//` |
| **mul**, **rmul** | Умножение | `*` |
| **pow**, **rpow** | Возведение в степень | `**` |

Точно так же, выполняя переиндексацию объекта Series или DataFrame, можно указать восполняемое значение:

In [73]:
df1.reindex(columns=df2.columns, index=df2.index, fill_value=0)

,a,b,c,d,e
0,0,1,2,3,0
1,4,5,6,7,0
2,8,9,10,11,0
3,0,0,0,0,0


#### Операции между DataFrame и Series
Как и в случае с массивами NumPy, арифметические операции между DataFrame и Series корректно определены. 

In [74]:
arr = np.arange(12).reshape(3, 4)
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [77]:
arr - arr[0]

array([[0, 0, 0, 0],
       [4, 4, 4, 4],
       [8, 8, 8, 8]])

In [ ]:
frame = pd.DataFrame(np.arange(12).reshape(4, 3),
                     columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
print(frame)

series = frame.iloc[0]
print('-' * 12)
print(series)

        b   d   e
Utah    0   1   2
Ohio    3   4   5
Texas   6   7   8
Oregon  9  10  11
------------
b    0
d    1
e    2
Name: Utah, dtype: int64
